# Impyte Documentation
This is a first practicle attempt to clarify the usage of `impyte`. It's a collection of easy applicable and reproducible examples that you could use to simplyify your data processing workflow.

## Importing and generating sample data
In order to show some of the features of the library, we'll be using Kaggle's HR data that can be found [here](https://www.kaggle.com/ludobenistant/hr-analytics).

In [575]:
# import library and data set
import impyte
import pandas as pd
from tools.data_prep import remove_random

In [576]:
data = pd.read_csv('data/hr_test.csv')

In [577]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Add random missing values
In order to show some of the pattern visualization and imputation methods, we need incomplete data sets. To achieve this, we'll be using a helper function that deletes values at random. 25 % of all values from each category will be deleted.

In [578]:
data = remove_random(data, .25)

In [579]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.87,6.0,265.0,5.0,0.0,1.0,0.0,sales,medium
1,NaN,0.48,4.0,220.0,3.0,1.0,0.0,0.0,support,low
2,NaN,0.92,6.0,164.0,4.0,1.0,0.0,0.0,marketing,low
3,NaN,0.99,5.0,176.0,4.0,1.0,0.0,0.0,sales,low
4,NaN,0.61,5.0,174.0,6.0,0.0,0.0,0.0,sales,medium


In [580]:
data.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000
mean,0.612652,0.716230,3.799521,201.056615,3.496889,0.145094,0.238359,0.021060
std,0.248861,0.171424,1.231321,49.933518,1.457834,0.352208,0.426094,0.143589
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [581]:
len(data)

14999

In [598]:
reload(impyte)

<module 'impyte' from 'impyte.pyc'>

## Testing rudimentary features
Below is a first trial run of some of the features implemented by impyter and its helper classes.

## `NanChecker`
Functionality testing of `NanChecker` class.

In [583]:
nan_checker = impyte.NanChecker()

#### `NanChecker.is_nan(data, nan_vals=None, recursive=True)`
Detect missing values (NaN in numeric arrays, empty strings in string arrays). NaN detection feature also enables NaN-value ingestion as parameter values.

In [584]:
# [True, False, False]
print nan_checker.is_nan(["", 'None', 'NaN'])

# [True, True, True]
print nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

[True, False, False]
[True, True, True]


In [585]:
# Recursive nan detection
# [True, True, False, [False, True, True]]
print nan_checker.is_nan(["", None, 'NaN', ["List Value 1", '', None]])

[True, True, False, [False, True, True]]


In [586]:
# Values can be declared as nan-values
# [True, False, False, True]
nan_checker.is_nan(['NaN', 'Empty', 'None', 'N/A'], nan_vals=['NaN', 'N/A'])

[True, False, False, True]

## `Pattern`
Functionality testing of `Pattern` class. The `Pattern` class stores different patterns and data summaries regarding NaN values. 

#### `Pattern._get_discrete_and_continuous(tmpdata)`
Returns the column names of discrete and continuous variables. Column names are stored in lists for easy selection. Those lists are stored in one dictionary object. All continuous column names can be accessed through `['continuous']` all discrete variables through `['discrete']`.

In [587]:
reload(impyte)
pattern_log = impyte.Pattern()

In [588]:
pattern_log._get_discrete_and_continuous(data)

{'continuous': ['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company',
  'Work_accident',
  'left',
  'promotion_last_5years'],
 'discrete': ['sales', 'salary']}

#### `Pattern._compute_pattern(data, nan_values="", verbose=False)`
Function that checks for missing values and prints out a quick table of a summary of missing values. Includes pattern overview and counts of missing values by column. Computes pattern and returns its pattern structure plus the count of data points for each of these patterns. To get a nice digestible table add `["table"]` selector to output.

In [589]:
pattern_dict = pattern_log._compute_pattern(data)
indices, table = pattern_dict["indices"], pattern_dict["table"]

In [590]:
table

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [591]:
indices[0][:10] # first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

In [592]:
pattern_nr = 0
print "Pattern {} has {:,} rows.".format(pattern_nr, len(indices[0])) 

Pattern 0 has 11,259 rows.


#### `Pattern.get_pattern()`
Returns NaN-patterns based on primary computation or initiates new computation of NaN-patterns. Uses `_compute_patterns()` to visualize patterns if not yet computed. Otherwise returns stored information regarding patterns.

In [593]:
pattern_log.get_pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


#### `Pattern.row_nan_pattern(row)`
This is a core piece to determining which patterns are available. Other methods work with the output of `row_nan_pattern()` to determine unique pattern structures and count these before turning them into readable tables.

In [594]:
# (1, 1, 'NaN', 'NaN')
print pattern_log.row_nan_pattern(['Value 1', 'Value 2', '', None])

(1, 1, 'NaN', 'NaN')


#### `Pattern.get_pattern_indices(pattern_no)`

In [595]:
pattern_log.get_pattern_indices(0)[:10] # get first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

#### `Pattern.get_continuous()`
Returns list with names of all continuous variables.

In [596]:
pattern_log.get_continuous()

['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years']

#### `Pattern.get_discrete()`
Returns list with names of all discrete variables.

In [597]:
pattern_log.get_discrete()

['sales', 'salary']

## `Impyter`

In [599]:
imputer = impyte.Impyter() # instantiate impyte class

### Load data into imputer

In [642]:
reload(impyte)
_ = imputer.load_data(data)

In [643]:
%timeit imputer.pattern()

The slowest run took 78710.71 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 12.2 µs per loop


In [644]:
%timeit imputer.get_pattern(4)

The slowest run took 6.17 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 782 µs per loop


In [645]:
reload(impyte)
imputer = impyte.Imputer(data) # instantiate impyte class
nan_checker = impyte.NanChecker()
pattern_log = impyte.Pattern()
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [646]:
_ = imputer.drop_pattern(4, inplace=True)

In [647]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374
10,1,1,1,1,1,1,NaN,1,1,1,374


In [648]:
_ = imputer.load_data(data)

In [649]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [650]:
_ = imputer.drop_pattern(4)

In [651]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [652]:
_ = imputer.drop_pattern(7, inplace=True)

In [654]:
imputer.pattern_log.store_tuple_colums

{(1, 1, 1, 1, 1, 1, 1, 1, 1, 1): [],
 (1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN'): [9],
 (1, 1, 1, 1, 1, 1, 1, 1, 'NaN', 1): [8],
 (1, 1, 1, 1, 1, 1, 1, 'NaN', 1, 1): [7],
 (1, 1, 1, 1, 1, 1, 'NaN', 1, 1, 1): [6],
 (1, 1, 1, 1, 1, 'NaN', 1, 1, 1, 1): [5],
 (1, 1, 1, 1, 'NaN', 1, 1, 1, 1, 1): [4],
 (1, 1, 1, 'NaN', 1, 1, 1, 1, 1, 1): [3],
 (1, 1, 'NaN', 1, 1, 1, 1, 1, 1, 1): [2],
 (1, 'NaN', 1, 1, 1, 1, 1, 1, 1, 1): [1],
 ('NaN', 1, 1, 1, 1, 1, 1, 1, 1, 1): [0]}

In [656]:
data.iloc[:, imputer.pattern_log.store_tuple_colums[(1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN')]].columns

Index([u'salary'], dtype='object')

In [564]:
data.tail()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
14994,0.77,0.68,5.0,162.0,4.0,1.0,0.0,0.0,technical,medium
14995,0.42,0.69,3.0,126.0,2.0,0.0,0.0,0.0,sales,low
14996,0.14,0.57,6.0,275.0,5.0,0.0,0.0,0.0,sales,low
14997,0.63,0.64,4.0,178.0,3.0,0.0,0.0,0.0,product_mng,low
14998,0.90,0.85,3.0,158.0,7.0,0.0,0.0,0.0,marketing,medium


### Timing some of the functions
Below is a piece of information on efficiency of the functions. This doesn't reflect complexity of the functions, but it will show an runtime estimate for your local machine.

In [565]:
%timeit nan_checker.is_nan(["", 'None', 'NaN'])
%timeit nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

100000 loops, best of 3: 5.01 µs per loop
100000 loops, best of 3: 4.95 µs per loop
